# Step Input

### Input files

The input of SoS step follows the following rules:

* **the input of a SoS step is by default the output of the previous step**, which is `None` for the first step.
* **`input` option**, which could be **a list** of filenames (string literal, variables, or expressions that return filenames).  Wildcard characters (`*`, which matches everything and `?`, which matches any single character) are acceptable. Nested lists are flattened.

Examples of input specification are as follows:

```
input: []

input: 'file1.fasta', 'file2.fasta'

input: 'file*.fasta', 'filename with space.fasta'

input:
    'file*.txt',
    'directory/file2.txt'

input: aligned_reads

input: aligned_reads, reference_genome

input: aligned_reads[2:]

input: 'data/*.fastq'

input: '*/GXT*.fastq'

input: func(parameter)
```

It is worth noting that

* The first examples shows that the step does not need any input file (so it does not depend on any other step).
* It does not matter if `aligned_reads` and `reference_genome` are strings or lists of strings because SoS will flatten nested lists to a single list of filenames.
* The `input` option tries to expand filenames with wildcard characters (`*` and `?`). This can be very useful for workflows that, for example, regularly scan a directory and process unprocessed files. However, because the value of this step depends on availability of files, the output of `sos show script` and the execution path will be unpredictable, and even wrong if there is no available file during the execution of `sos show script`.

The input files will be evaluated and form a list of input files. They are by default sent to the step process all at once as varible `_input`, but can also be sent in groups, each time
with different `_input`. Here `_input` is a temporary variable that is available only within the step.

### Option `filetype`

SoS allows the specification of input options, which are appended to input file list as comma separated `name=value` pairs.

Option `filetype` accepts one or more filetypes or a lambda function. For example,

```
[step]
input:
	input_files, filetype='*.fastq'
```

passes only files with extension `*.fastq`.

```
[step]
input:
	input_files,
	filetype=['*.fastq', '*.fastq.gz']

```

passes only files with extension `.fastq` or `.fastq.gz`. Under the hood SoS treats the pattern as Unix shell-stype wildcard pattern (with `*`, `?`, `[seq]` and `[!seq]`, see [doc](https://docs.python.org/2/library/fnmatch.html#module-fnmatch) for details) so

* **`filetype='.txt'` does not match `file.txt`**
* `filetype='*.fastq*'` matches `a.fastq`, `a.fastq.gz` and `a.fastq.zip`
* `filetype='[!_]*.txt'` matches `file1.txt` but not `_file1.txt`

If you need more refined control over the selection of files, you can use lambda functions (a bit python knowledge is required). For example,

```
[step]
input:
	input_files,
	filetype=lambda x: open(x).readline().startswith('##fileformat=VCF4.1')
```

passes only files with the first line starting with string `##fileformat=VCF4.1``.

### Option `group_by`

SoS by default passes all input files to step process as a single list. Option `group_by` pass input files in groups, each time with a subset of input files named `_input`. SoS allows you to group input by `single` (individual file), `pairs` (match first half of files with the second half),  `combinations` (all unordered combinations of 2-sets), `pairwise` (all adjacent 2-sets), or chunks of size `N` (integer `group_by=3` or string `group_by='4'`). For example, with the following sos script

```
# how to group input files
parameter: group = 'all'

[0]
print('group_by=${group}')
input: 'file1', 'file2', 'file3', 'file4', group_by=group
print('${_index}: ${_input}')
```

The output of commands are

```
$ sos run test.sos  -i -v0
group_by=all
0: file1 file2 file3 file4

$ sos run test.sos --group 'single' -i -v0
group_by=single
0: file1
1: file2
2: file3
3: file4

$ sos run test.sos --group 'pairwise' -i -v0
group_by=pairwise
0: file1 file2
1: file2 file3
2: file3 file4

$ sos run test.sos --group 'pairs' -i -v0
group_by=pairs
0: file1 file3
1: file2 file4

$ sos run test.sos --group 'combinations' -i -v0
group_by=combinations
0: file1 file2
1: file1 file3
2: file1 file4
3: file2 file3
4: file2 file4
5: file3 file4

$ sos run test.sos --group 1 -i -v0
group_by=1
0: file1
1: file2
2: file3
3: file4

$ sos run test.sos --group 2 -i -v0
group_by=2
0: file1 file2
1: file3 file4
```
Here we are running the script in inspect mode to avoid "file not found" errors for input files.

Obviously, the output of the `pairs` cases depends on the order of files. If you need to pair files in any particular order, you can control it in input. For example

```
[step]
input:
	sorted([x for x in fastq_files if '_R1_' in x]),
	sorted([x for x in fastq_files if '_R2_' in x]),
	group_by='pairs'

run('echo ${input}')
```

will take all input files and sort them by `_R1_` and `_R2_` and by filename, and pair them.

### Option `for_each`

Option `for_each` allows you to repeat step process for each value of a variable. For example,

```
[0]
method = ['m1', 'm2']
input: 'file1', 'file2', for_each='method'
print('${_index}: ${_input} ${_method}')
```

will repeat the step with each item of variable `method`

```
$ sos run test.sos -v0
0: file1 file2 m1
1: file1 file2 m2
```

Note that the SoS automatically creates a loop variable `_method` for variable `method`. If the variable is an attribute of an object (e.g. `aligned.output`), the iterator variable will be named without the attribute part.

Nested loops are also allowed. For example,

```python
[0]
method = ['m1', 'm2']
pars = [1, 2]
input: 'file1', 'file2', for_each=['method', 'pars']
print('${_index}: _input=${_input} _method=${_method}, _pars=${_pars}')
```

would produce

```
$ sos run test.sos -v0
0: _input=file1 file2 _method=m1, _pars=1
1: _input=file1 file2 _method=m2, _pars=1
2: _input=file1 file2 _method=m1, _pars=2
3: _input=file1 file2 _method=m2, _pars=2
```

If you would like to loop the process with several parameters, you can put them into the same level by 'var1,var2'. For example,


```python
[0]
method = ['m1', 'm2']
pars = [1, 2]
input: 'file1', 'file2', for_each='method,pars'
print('${_index}: _input=${_input} _method=${_method}, _pars=${_pars}')
```

would produce

```
$ sos run test.sos -v0
0: _input=file1 file2 _method=m1, _pars=1
1: _input=file1 file2 _method=m2, _pars=2
```

The variable passed to option `for_each` can a sequence, or a Pandas dataframe. In this case, each `_loop` variable presents a line in the dataframe and you can access single values using format `_loop["header"]`. For example

```
[0]
import pandas as pd
data = pd.DataFrame([(1, 2, 'Hello'), (2, 4, 'World')], columns=['A', 'B', 'C'])
input: for_each='data'
output: '${_data["A"]}_${_data["B"]}_${_data["C"]}.txt'
sh:
    touch ${_output}
```

would produce results with names `1_2_Hello.txt` and `2_4_World.txt`.

### Option `paired_with`

Input files might might come with additional information such as sample type, and sample name, and you can pair these information to input files using the `paired_with` option. For example, `bam_files` in the following example have matched `mutated` and `sample_name`, you can atteched these information to groups of input files `_input` with looped paired with variables.

```
[0]
bam_files = ['case/A1.bam', 'case/A2.bam', 'ctrl/A1.bam', 'ctrl/A2.bam']
mutated = ['case', 'case', 'ctrl', 'ctrl']
sample_name = ['A1', 'A2', 'A1', 'A2']

input: bam_files, paired_with=['mutated', 'sample_name'], group_by='pairs'

print('${_index}: _input=${_input} _mutated=${_mutated}, _sample_name=${_sample_name}')
```

Output:

```
$ sos run test.sos -v0
0: _input=case/A1.bam ctrl/A1.bam _mutated=case ctrl, _sample_name=A1 A1
1: _input=case/A2.bam ctrl/A2.bam _mutated=case ctrl, _sample_name=A2 A2
```

Similar to option `for_each`, if the variable is an attribute of an object (e.g. `aligned.output`), the iterator variable will be named without the attribute part (e.g. `_aligned` for `paired_with='aligned.output'`).

### Option `pattern`

This option does the reverse of function `expand_pattern`. It uses named wildcards to match pattern to all input files, and creates step variables for these wildcard objects. For example,

```python
[step]
input:  'a-20.txt', 'b-10.txt', pattern = '{name}-{par}.txt'
output: expand_pattern('{name}-processed-{par}.txt')
run('echo ${output}; touch ${output}')
```

will take all input files and extract `name` and `par` from each file name as variables `name` and `par`. It is then used to create output file names adding the word `processed` in between these wildcard objects. The outcome of the SoS script above is creation of files `a-processed-10.txt` and `b-processed-20.txt`.

When wildcard objects are accessed as step variables, both variable names with and without `_` prefix is available, e.g. in this example, both `_name` and `name`, `_par` and `par` are avaiable and are the same. The two conventions will only differ when [`group_by`](Documentation#option-group_by) or ['for_each'](Documentation#option-for_each) is also used.

```python
[step]
input:  'a-20.txt', 'b-10.txt', extract = '{name}-{par}.txt', group_by='single'
output: expand = '{_name}-processed-{_par}.txt'
run('echo ${_output}; touch ${_output}')
```

Please note that the sigil `{}` is exclusively used for named wildcards and you should not use `${}` because it will be interpreted as strings.

### Option `skip`

Option `skip` takes either a constant (`True` or `False`) or a function. Option `skip=True`  will make SoS skip the execution of the current step. Using `skip=True` is not very useful so this option is often used with a SoS variable. For example

```python
[10]
input:
	fasta_files,
	skip=len(fasta_failes) == 1

output: 'merged.fasta'

run('command to merge multiple fasta files.')

```

Here the `skip` option gets the value of `True` if there is only one input file. The command to merge multiple input files would then be skipped.

Another use of option `skip` is to assign it a function. In this case, this function will be applied
to each input group with `_input` as the first variable and all `paired_with`, `pattern`, `loop` variables (e.g. `_loopvar`) as keyword arguments. The input group will be skipped if this function returns `False`. For example,

```
[10]
def filter_group(ifiles, **kwargs):
    return all(os.path.getfile(x) > 0 for x in ifiles)

input: group_by='combinations', skip=filter_group
```

will check all input groups and skip groups with one or two empty files (file size = 0).


### `dynamic` input files

In order to determine the best execution strategy, SoS evaluates all expressions for all steps before the execution of a workflow to figure
out input and output of steps. This works most of the time but sometimes the input of a step can only be determined at runtime. For example,
if you would like your workflow to automatically scan an input directory and process all fasta files under it, or if a previous step produces
files that cannot be determined beforehand, you can specify input files as follows,

```python
input: 'input/*.fasta'
```

The problem is that no file or a wrong set files might exist during the planing stage so SoS might skip this step or start the step
with a wrong set of files. To address this problem, you can declare the input files as **dynamic** by passing a `dynamic` object

```python
input: dynamic('input/*.fasta')
```

This tells SoS that the input of this step can only be determined at runtime and will execute the step only after all its previous
steps have been completed.

### Summary

Options of step `input` are evaluated in the following orders:

1. A list of input files, if specified, would replace `input`, which is by default output from the previous step.
2. Option `filetype` filters input files. **The output becomes `input`**.
3. Option `group_by` groups the files into several groups, named `_input`
4. Option `for_each` repeat `_input` for each loop var, named `_loopvar` if `for_each='loopvar'`.
5. Option `paired_with` pairs one or more variables with `input`, variable `paired` is paired with `input`
    and variable `_paired` is paired with `_input` in each loop if `paired_with='paired'`
6. Option `pattern` extract variables from filenames in `input`. Variable `extracted` is paired with `input`
    and variable `_extracted` is paired with `_input` in each loop if `extract='{extracted}_other_part'`.
7. Option `skip` optionally skip all or part of the input groups.

The differences between looped and non-loop steps are sumarized in the following figure

![step_loop](media/step_loop.jpg)